In [2]:
import sqlite3
import pandas as pd

In [3]:
conn = sqlite3.connect('dannys_diner.db')

#### 1. What is the total amount each customer spent at the restaurant?

In [13]:
query ="""
SELECT
    s.customer_id,
    SUM(menu.price) AS "Total Spent"

FROM sales AS s

LEFT JOIN menu
ON s.product_id = menu.product_id

GROUP BY customer_id
"""

df_total_spent = pd.read_sql_query(query, conn)
df_total_spent.head()

,customer_id,Total Spent
0,A,76
1,B,74
2,C,36


#### 2. How many days has each customer visited the restaurant?

In [12]:
query ="""
SELECT
    customer_id,
    COUNT(DISTINCT(order_date)) AS visit_count

FROM sales

GROUP BY customer_id
"""

df_days = pd.read_sql_query(query, conn)
df_days.head()

,customer_id,visit_count
0,A,4
1,B,6
2,C,2


#### 3. What was the first item from the menu purchased by each customer?

In [11]:
query ="""
SELECT
    s.customer_id,
    menu.product_name

FROM sales AS s

JOIN menu
ON s.product_id = menu.product_id

WHERE s.order_date = (
    SELECT MIN(order_date)
    FROM sales
    WHERE customer_id = s.customer_id
)

GROUP BY s.customer_id;
"""

df_days = pd.read_sql_query(query, conn)
df_days.head()

,customer_id,product_name
0,A,sushi
1,B,curry
2,C,ramen


#### 4. What is the most purchased item on the menu and how many times was it purchased by all customers?


In [10]:
query ="""
SELECT
    menu.product_name,
    COUNT(product_name) AS "Number of Sales"

FROM sales AS s

LEFT JOIN menu
ON s.product_id = menu.product_id

GROUP BY menu.product_name

ORDER BY "Number of Sales" DESC

LIMIT 1;
"""

df_most_purchased_item = pd.read_sql_query(query, conn)
df_most_purchased_item.head()

,product_name,Number of Sales
0,ramen,8


#### 5. Which item was the most popular for each customer?

In [15]:
query ="""
WITH item_count AS (
    SELECT
        s.customer_id,
        menu.product_name,
        COUNT(s.product_id) AS purchase_count,
        RANK() OVER
        (PARTITION BY s.customer_id ORDER BY COUNT(s.product_id) DESC) AS rank
    FROM sales AS s
    
    LEFT JOIN menu
    ON s.product_id = menu.product_id

    GROUP BY s.customer_id, menu.product_name
)

SELECT
    customer_id,
    product_name,
    purchase_count
FROM item_count
WHERE rank = 1
"""

df_most_popular_item = pd.read_sql_query(query, conn)
df_most_popular_item.head()

,customer_id,product_name,purchase_count
0,A,ramen,3
1,B,sushi,2
2,B,ramen,2
3,B,curry,2
4,C,ramen,3


#### 6. Which item was purchased first by the customer after they became a member?

In [19]:
query ="""
WITH member_purchases AS (

    SELECT
        s.customer_id,
        s.order_date,
        mn.product_name,
        mb.join_date,
        RANK() OVER (PARTITION BY s.customer_id ORDER BY s.order_date ASC) AS rank

    FROM sales AS s
    
    LEFT JOIN members AS mb
    ON s.customer_id = mb.customer_id

    LEFT JOIN menu as mn
    ON s.product_id = mn.product_id

    WHERE s.order_date >= mb.join_date
)

SELECT
    customer_id,
    product_name,
    order_date
FROM member_purchases
WHERE rank = 1
"""

df_after_member = pd.read_sql_query(query, conn)
df_after_member.head()

,customer_id,product_name,order_date
0,A,curry,2021-01-07
1,B,sushi,2021-01-11


#### 7. Which item was purchased just before the customer became a member?

In [21]:
query ="""
WITH before_member AS (  
    
    SELECT
        s.customer_id,
        s.order_date,
        s.product_id,  
        mb.join_date,
        RANK() OVER (PARTITION BY s.customer_id ORDER BY s.order_date DESC) AS rank
    
    FROM sales AS s
    
    JOIN members AS mb  
    ON s.customer_id = mb.customer_id
    
    WHERE s.order_date < mb.join_date
)

SELECT
    bm.customer_id,
    mn.product_name,
    bm.order_date
FROM before_member AS bm

JOIN menu AS mn  
ON bm.product_id = mn.product_id

WHERE bm.rank = 1;

"""

df_before_member = pd.read_sql_query(query, conn)
df_before_member.head()

,customer_id,product_name,order_date
0,A,sushi,2021-01-01
1,A,curry,2021-01-01
2,B,sushi,2021-01-04


- 8. What is the total items and amount spent for each member before they became a member?
- 9.  If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?
- 10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?